## 简单PDF文本提取

In [11]:
# 初始化 PDF文件路径
file_path = "../../90-文档_Data/黑神话/黑神话悟空.pdf"

1. 使用PyPDF工具

In [12]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path)
pages = loader.load()
print(f"加载了 {len(pages)} 页PDF文档")
for page in pages:
    print(page.page_content)

加载了 5 页PDF文档
08.20
直面天命
序章
《
黑神话：悟空
》
是一款基于
《
西游记
》
改编的中国神话动作角色扮演游戏，玩家化身
“天命之人”，在险象环生的西游冒险中追寻传说背后的秘密。
改编自公元
1592
年中国神魔小说
《
西游记
》
启
程
B L A C K M Y T H
第
壹
章
关卡多端
雄奇壮丽，光怪陆离
重走西游故地，再写神话结局
W U K O N G
启程
 无惧
 历练
 逢缘
天
命
你
,
也想当神仙
?
天命人
T I A N
M I N G
世
人
都
晓
神
仙
好
唯
有
功
名
忘
不
了
古
今
将
相
在
何
方
荒
冢
一
堆
草
末
了
启程
 无惧
 历练
 逢缘


2. 使用PyMuPDF工具

    PyMuPDF (fitz) 与 Unstructured 对比:
    - 优势:
        1. 更快的处理速度
        2. 更细粒度的PDF控制能力
        3. 可以获取更多元数据和文档结构信息
        4. 内存占用更少
        5. 不依赖外部工
    - 劣势:
        1. 文本提取的智能化程度较低
        2. 没有自动的文档结构理解
        3. 需要手动处理布局分析

In [13]:
import fitz

# 打开PDF文件
doc = fitz.open(file_path)
text = [page.get_text() for page in doc]
print(text)

# 示例: 使用PyMuPDF的基础功能
print("=== PyMuPDF 基本信息提取 ===")
print(f"文档页数: {len(doc)}")
print(f"文档标题: {doc.metadata['title']}")
print(f"文档作者: {doc.metadata['author']}")
print(f"文档元数据: {doc.metadata}")  # 比Unstructured提供更多元数据

# 遍历每一页
for page_num, page in enumerate(doc):
    # 提取文本
    text = page.get_text()
    print(f"\n--- 第{page_num + 1}页 ---")
    print("文本内容:", text[:200])  # 显示前200个字符
    
    # 提取图片
    images = page.get_images()
    print(f"图片数量: {len(images)}")
    
    # 获取页面链接
    links = page.get_links()
    print(f"链接数量: {len(links)}")
    
    # 获取页面大小
    width, height = page.rect.width, page.rect.height
    print(f"页面尺寸: {width} x {height}")

doc.close()

['08.20\n直面天命\n', '序章\n《黑神话：悟空》是一款基于《西游记》改编的中国神话动作角色扮演游戏，玩家化身\n“天命之人”，在险象环生的西游冒险中追寻传说背后的秘密。\n', '改编自公元1592年中国神魔小说《西游记》\n', '启程\nB\nL\nA\nC\nK\nM\nY\nT\nH\n第\n壹\n章\n关卡多端\n雄奇壮丽，光怪陆离\n重走西游故地，再写神话结局\nW\nU\nK\nO\nN\nG\n启程\n无惧\n历练\n逢缘\n', '天命\n你,也想当神仙?\n天命人\nT\nI\nA\nN\nM\nI\nN\nG\n世\n人\n都\n晓\n神\n仙\n好\n唯\n有\n功\n名\n忘\n不\n了\n古\n今\n将\n相\n在\n何\n方\n荒\n冢\n一\n堆\n草\n末\n了\n启程\n无惧\n历练\n逢缘\n']
=== PyMuPDF 基本信息提取 ===
文档页数: 5
文档标题: 
文档作者: 风 听
文档元数据: {'format': 'PDF 1.7', 'title': '', 'author': '风 听', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® for Microsoft 365', 'producer': 'Microsoft® PowerPoint® for Microsoft 365', 'creationDate': "D:20241107154241+08'00'", 'modDate': "D:20241107154241+08'00'", 'trapped': '', 'encryption': None}

--- 第1页 ---
文本内容: 08.20
直面天命

图片数量: 10
链接数量: 0
页面尺寸: 960.0 x 540.0

--- 第2页 ---
文本内容: 序章
《黑神话：悟空》是一款基于《西游记》改编的中国神话动作角色扮演游戏，玩家化身
“天命之人”，在险象环生的西游冒险中追寻传说背后的秘密。

图片数量: 15
链接数量: 0
页面尺寸: 960.0 x 540.0

--- 第3页 ---
文本内容: 改编自公元1592年中国神魔小说《西游记》

图片数量: 8
链接数量: 0
页

3. 使用 pytesseract + pdf2image  处理扫描图片型的PDF  
```bash
    sudo apt-get install tesseract-ocr
    sudo apt-get install tesseract-ocr-chi-sim
````

In [15]:
import pdf2image
import pytesseract
import os

output_dir = "temp_images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 将 PDF 转换为图片并保存
images = pdf2image.convert_from_path(file_path)
image_paths = []
for i, image in enumerate(images):
    image_path = os.path.join(output_dir, f'page_{i+1}.png')
    image.save(image_path)
    image_paths.append(image_path)

# 使用 pytesseract 提取文本
for i, image in enumerate(images):
    text = pytesseract.image_to_string(image, lang='chi_sim')
    print(f"第 {i+1} 页文本:")
    print(text)
    print("\n") 

# 清理临时文件
for image_path in image_paths:
    os.remove(image_path)
os.rmdir(output_dir)

第 1 页文本:
山

，申 国-歼 事

GAME SCIENCE

 游戏科学

 

 

08. 20

直面天

避



第 2 页文本:
 游戏科学
GAME SCIENCE

世界虽医， 中国故事

ev

 

和

可

 

 

二

|

 

 

 

 

二

 

 

 

 

 

 

 

 

 

国神话动作角色扮演游戏，玩家化身

 

二二人汪症生生 帮SA这

 

册



第 3 页文本:
 



第 4 页文本:
 

 

 

 

雄奇壮丽，光怪陆离
本

 

 

 

 

 

 
 

 

1

 

多

SS  有全
和
Re 0  区
To

下     人 SS

1

归

 

 

 

 

 



第 5 页文本:
二
一你,也想当神仙?

 

 

 

局

|

六

用

放 忌 过二-玫

妆证

二

“ 法 主盖库

棋
-二

好



